In [2]:
!kill -9 32762

In [3]:
!nvidia-smi

Thu Apr 26 18:06:53 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.30                 Driver Version: 390.30                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:01:00.0  On |                  N/A |
|  0%   48C    P2    44W / 200W |    598MiB /  8116MiB |     11%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [14]:
import os
checkpoint = 'output/shallowunet/1'
frozen_dir = checkpoint+'/frozen'
os.system('rm -r {}'.format(frozen_dir))
cmd = 'python shalowunet.py --mode export \
    --checkpoint {} --crop_size 512 --output_dir {}'.format(checkpoint, frozen_dir)
print(cmd)
os.system(cmd)

python shalowunet.py --mode export     --checkpoint output/shallowunet/1 --crop_size 512 --output_dir output/shallowunet/1/frozen


0

In [15]:
from time import time
import tensorflow as tf
import cv2
import numpy as np
import os
from glob import glob
import math


def get_tensor_by_name(name):
    name_on_device = '{}:0'.format(name)
    return tf.get_default_graph().get_tensor_by_name(name_on_device)

def load_image(path, verbal=False):
    print('---------------------------\nprocess:', path)
    name = path.split('/')[-1].split('.')[0]
    image = cv2.imread(path)
    p = 286/256
    image = cv2.resize(image, (0,0), fx=p*0.3, fy=p*0.3)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def write(path, image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return cv2.imwrite(path, image)
print('Finish')
def get_name(path):
    name, _ = os.path.splitext(os.path.basename(path))
    return name   

Finish


In [16]:
def save_output(image, output_image, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    inp = image
    out1 = output_image[:,:,0]
    out2 = output_image[:,:,-1]
    #cv2.imwrite(os.path.join(save_dir, 'input.png'), inp)
    write(os.path.join(save_dir, 'input.png'), inp)
    cv2.imwrite(os.path.join(save_dir, 'output1.png'), 255-out1)
    cv2.imwrite(os.path.join(save_dir, 'output2.png'), 255-out2)
    print('Output: ', save_dir)

In [17]:
def run(sess, feed_image, batch_size=8):
    batch_input = sess.run(batch_input_tensor, {inputs:feed_image})
    rv = []
    
    for i in range(0, len(batch_input), batch_size):
        print('\r {:0.2f} %'.format(i/len(batch_input)), end='')
        rv.append(sess.run(batch_output_tensor, {batch_input_placeholder: batch_input[i:i+batch_size]}))
    output_feed = np.concatenate(rv, axis=0)
    return sess.run(outputs, {batch_output_placeholder: output_feed, inputs:feed_image})

# RESTORE MODEL


In [18]:
input_dir = 'data/test_data/'
checkpoint = frozen_dir
output_dir = os.path.join(checkpoint, 'images')


tf.reset_default_graph()
from time import time
meta_path = os.path.join(checkpoint, 'export.meta')
print('meta path:', meta_path)

assert os.path.exists(meta_path), meta_path+' does not exist'
tf.train.import_meta_graph(meta_path)
saver = tf.train.Saver()

meta path: output/shallowunet/1/frozen/export.meta


# RUN SAMPLE

In [19]:
print('input_dir:', input_dir)
paths = glob('{}/*.JPG'.format(input_dir))+glob('data/scaned/*.JPG')
# paths = [path for path in paths if get_name(path).isdigit()]
paths = ['xxx.jpg']

input_dir: data/test_data/


In [20]:
with tf.Session() as sess:

    saver.restore(sess, tf.train.latest_checkpoint(
        checkpoint))

    parameter_count = tf.reduce_sum([tf.reduce_prod(tf.shape(v)) for v in tf.trainable_variables()])
    print('num of params:', sess.run(parameter_count))


    os.makedirs(output_dir, exist_ok=True)
    start = time()

    assert len(paths) > 0
    print('Num of sample:', len(paths), input_dir)
    inputs = get_tensor_by_name('inputs')
    outputs = get_tensor_by_name('outputs')
    batch_input_tensor = get_tensor_by_name('batch_input_tensor')
    batch_input_placeholder = get_tensor_by_name('batch_input_placeholder')
    batch_output_tensor = get_tensor_by_name('batch_output_tensor')
    batch_output_placeholder = get_tensor_by_name('batch_output_placeholder')
    start = time()
    begin = start
    for path in paths:
        name = path.split('/')[-1].split('.')[0]
        image = load_image(path, verbal=True)
        print('image:', path, '\t shape: ', image.shape)
        output_image = run(sess, image)
        save_dir = os.path.join(checkpoint, 'temp1/{}'.format(name))
        save_output(image, output_image, save_dir)
        print('Running time:', time()-start)


    print('Running time:', time()-begin)


INFO:tensorflow:Restoring parameters from output/shallowunet/1/frozen/export
num of params: 1043529
Num of sample: 1 data/test_data/
---------------------------
process: xxx.jpg
image: xxx.jpg 	 shape:  (1351, 1014, 3)
 0.67 %Output:  output/shallowunet/1/frozen/temp1/xxx
Running time: 2.120087146759033
Running time: 2.1201066970825195
